In [1]:
import sqlite3
import os
import numpy as np

In [2]:
conn = sqlite3.connect('test.sqlite')

In [3]:
c = conn.cursor()

In [4]:
c.execute("DROP TABLE IF EXISTS customer")
c.execute("DROP TABLE IF EXISTS department")
conn.commit()

#### Создание таблицы

In [5]:
c.execute("CREATE TABLE customer (name TEXT, id INTEGER)")

c.execute("CREATE TABLE department (title TEXT, employee_id INTEGER)")

conn.commit()

#### Добавление данных в таблицу

In [6]:
names = ["Ivan", "Maria", "Olga", "Petr", "Sergey", "John", "Anton"]

In [7]:
departments = ["Бухгалтерия", "Продажи", "Разработка", "Производство", "Охрана", "Кадры", "АХО"]

In [8]:
for i in range(100):
    name_ = np.random.choice(names)
    dpt_ = np.random.choice(departments)
    id_ = i
    c.execute("INSERT INTO customer (name, id) VALUES ('{}', '{}')".format(name_, id_))
    
    c.execute("INSERT INTO department (title, employee_id) VALUES ('{}', '{}')".format(dpt_, id_))

In [9]:
conn.commit()

#### Получение данных из таблицы

In [10]:
c.execute("SELECT * from customer")

In [11]:
data = c.fetchall()

In [12]:
data[:10], len(data)

([('Sergey', 0),
  ('Ivan', 1),
  ('Petr', 2),
  ('Olga', 3),
  ('Anton', 4),
  ('Petr', 5),
  ('Sergey', 6),
  ('Maria', 7),
  ('Sergey', 8),
  ('Petr', 9)],
 100)

#### Все пользователи с id < 10

In [13]:
c.execute("SELECT * from customer WHERE id<10")
data = c.fetchall()
data, len(data)

([('Sergey', 0),
  ('Ivan', 1),
  ('Petr', 2),
  ('Olga', 3),
  ('Anton', 4),
  ('Petr', 5),
  ('Sergey', 6),
  ('Maria', 7),
  ('Sergey', 8),
  ('Petr', 9)],
 10)

#### Все пользователи с именем Sergey

In [14]:
c.execute("SELECT * from customer WHERE name='Sergey'")
data = c.fetchall()
data, len(data)

([('Sergey', 0),
  ('Sergey', 6),
  ('Sergey', 8),
  ('Sergey', 10),
  ('Sergey', 12),
  ('Sergey', 19),
  ('Sergey', 24),
  ('Sergey', 26),
  ('Sergey', 34),
  ('Sergey', 38),
  ('Sergey', 44),
  ('Sergey', 50),
  ('Sergey', 51),
  ('Sergey', 52),
  ('Sergey', 60),
  ('Sergey', 64),
  ('Sergey', 73)],
 17)

In [15]:
c.execute("SELECT name from customer WHERE name='Sergey'")
data = c.fetchall()
data, len(data)

([('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',),
  ('Sergey',)],
 17)

In [16]:
c.execute("SELECT DISTINCT name from customer WHERE name='Sergey'")
data = c.fetchall()
data, len(data)

([('Sergey',)], 1)

#### Найти пользователей с именем Maria с четными id

In [17]:
c.execute("SELECT DISTINCT * FROM customer WHERE name='Maria' AND id%2=0")
data = c.fetchall()
data, len(data)

([('Maria', 18),
  ('Maria', 42),
  ('Maria', 56),
  ('Maria', 66),
  ('Maria', 76),
  ('Maria', 90)],
 6)

In [18]:
c.execute("""
             SELECT DISTINCT * FROM customer
             WHERE name='Maria' AND id%2=0
          """)
          
data = c.fetchall()
data, len(data)

([('Maria', 18),
  ('Maria', 42),
  ('Maria', 56),
  ('Maria', 66),
  ('Maria', 76),
  ('Maria', 90)],
 6)

#### Найти имена сотрудников, работающих в разработке

In [19]:
c.execute("""
             SELECT DISTINCT customer.name, department.title FROM customer
             JOIN department ON customer.id=department.employee_id
             WHERE department.title='Разработка'
          """)
          
data = c.fetchall()
data, len(data)

([('Petr', 'Разработка'),
  ('Maria', 'Разработка'),
  ('Olga', 'Разработка'),
  ('John', 'Разработка'),
  ('Anton', 'Разработка'),
  ('Ivan', 'Разработка'),
  ('Sergey', 'Разработка')],
 7)

#### Найти имена сотрудников, работающих в разработке и АХО

In [20]:
c.execute("""
             SELECT DISTINCT customer.name, department.title FROM customer
             JOIN department ON customer.id=department.employee_id
             WHERE department.title='Разработка'
             UNION
             SELECT DISTINCT customer.name, department.title FROM customer
             JOIN department ON customer.id=department.employee_id
             WHERE department.title='АХО'
          """)
          
data = c.fetchall()
data, len(data)

([('Anton', 'АХО'),
  ('Anton', 'Разработка'),
  ('Ivan', 'АХО'),
  ('Ivan', 'Разработка'),
  ('John', 'АХО'),
  ('John', 'Разработка'),
  ('Maria', 'АХО'),
  ('Maria', 'Разработка'),
  ('Olga', 'АХО'),
  ('Olga', 'Разработка'),
  ('Petr', 'АХО'),
  ('Petr', 'Разработка'),
  ('Sergey', 'АХО'),
  ('Sergey', 'Разработка')],
 14)

#### Найти имена сотрудников, работающих в АХО, имена которых не совпадают с разработчиками

In [21]:
c.execute("""
             SELECT DISTINCT customer.name FROM customer
             JOIN department ON customer.id=department.employee_id
             WHERE department.title='АХО'
             EXCEPT
             SELECT DISTINCT customer.name FROM customer
             JOIN department ON customer.id=department.employee_id
             WHERE department.title='Разработка'
          """)
          
data = c.fetchall()
data, len(data)

([], 0)

#### Найти имя сотрудника с самым высоким id

In [22]:
c.execute("""
            SELECT * from customer
            WHERE id = (SELECT MAX(id) from customer)
          """)
data = c.fetchall()
data

[('Maria', 99)]

#### Найти среднее id сотрудников с именем Maria

In [23]:
c.execute("SELECT * FROM customer where name='Maria'")
data = c.fetchall()
data

[('Maria', 7),
 ('Maria', 11),
 ('Maria', 18),
 ('Maria', 27),
 ('Maria', 35),
 ('Maria', 42),
 ('Maria', 56),
 ('Maria', 66),
 ('Maria', 67),
 ('Maria', 69),
 ('Maria', 76),
 ('Maria', 79),
 ('Maria', 83),
 ('Maria', 90),
 ('Maria', 95),
 ('Maria', 99)]

In [24]:
c.execute("""
            SELECT AVG(id) from customer
            WHERE name='Maria'
          """)
data = c.fetchall()
data

[(57.5,)]

#### Найти количество сотрудников с именем Maria

In [25]:
c.execute("""
            SELECT COUNT(id) from customer
            WHERE name='Maria'
          """)
data = c.fetchall()
data

[(16,)]

#### Найти количество сотрудников с именем Ivan, работающих на производстве

In [26]:
c.execute("""
            SELECT * from customer
            JOIN department ON customer.id=department.employee_id
            WHERE customer.name='Ivan' and department.title='Производство'
          """)
data = c.fetchall()
data

[('Ivan', 29, 'Производство', 29),
 ('Ivan', 36, 'Производство', 36),
 ('Ivan', 41, 'Производство', 41),
 ('Ivan', 88, 'Производство', 88)]

In [27]:
c.execute("""
            SELECT COUNT(customer.name) from customer
            JOIN department ON customer.id=department.employee_id
            WHERE customer.name='Ivan' AND department.title='Производство'
          """)
data = c.fetchall()
data

[(4,)]

#### Имена, встречающиеся более 15 раз

In [28]:
c.execute("""
            SELECT name, COUNT(name), MAX(id), MIN(id) from customer
            GROUP BY name            
          """)
data = c.fetchall()
data

[('Anton', 11, 96, 4),
 ('Ivan', 14, 88, 1),
 ('John', 13, 98, 17),
 ('Maria', 16, 99, 7),
 ('Olga', 18, 94, 3),
 ('Petr', 11, 70, 2),
 ('Sergey', 17, 73, 0)]

In [29]:
c.execute("""
            SELECT name from customer
            GROUP BY name
            HAVING COUNT(name)>15
          """)
data = c.fetchall()
data

[('Maria',), ('Olga',), ('Sergey',)]

#### Показать имена сотрудников АХО, имена которых не совпадают с сотрудниками разработки

In [30]:
c.execute("""
            SELECT DISTINCT name FROM customer
            JOIN department ON customer.id=department.employee_id
            WHERE department.title='АХО'
          """)
data = c.fetchall()
data

[('Olga',),
 ('Petr',),
 ('Sergey',),
 ('John',),
 ('Ivan',),
 ('Maria',),
 ('Anton',)]

In [31]:
c.execute("""
            SELECT DISTINCT name FROM customer
            JOIN department ON customer.id=department.employee_id
            WHERE department.title='Разработка'
          """)
data = c.fetchall()
data

[('Petr',),
 ('Maria',),
 ('Olga',),
 ('John',),
 ('Anton',),
 ('Ivan',),
 ('Sergey',)]

In [32]:
c.execute("""
            SELECT DISTINCT name FROM customer          
            WHERE 
             id IN (SELECT employee_id FROM department WHERE title='АХО')
            AND
             NOT name IN (SELECT name FROM customer
                          JOIN department ON customer.id=department.employee_id
                          WHERE title='Разработка')
          """)
data = c.fetchall()
data

[]

#### Закрытие соединения с базой данных и удаление файла базы данных

In [33]:
conn.close()

In [34]:
if os.path.isfile('test.sqlite'):
    os.remove('test.sqlite')

In [35]:
!ls

 ARIMA.ipynb				  NQueensTask.ipynb
 digits_recognition_with_TF_DNN.ipynb	  PictureCapture.ipynb
 EDA.ipynb				  Pipelines.ipynb
 flask					  README.md
'GridSearchCV example.ipynb'		  RNN.ipynb
 Holdout_model_testing.ipynb		  SQL_DB.ipynb
 IMDB_emotions_with_TF_Embeddings.ipynb   TensorFlow_basics.ipynb
 LICENSE				  TextFeaturesExtraction.ipynb
 NetworkX.ipynb
